# Neural Machine Translation
#### Applied Text Mining - Utrecht Summer School

Welcome to the seventh practical of the course “Applied Text Mining”. 

In this practical, we will create models for neural machine translation. Today we are curious to see how a simple deep learning based model translates a sentence into its counterpart. See these examples:

<img src="translation_example.png">

<img src="translation_example2.png">

The objective from this practical is to convert a Dutch sentence to its English counterpart using a Neural Machine Translation (NMT) system. We will implement this task by building a simple Sequence-to-Sequence model with the help of Keras library.

Today we will use the following libraries. Take care to have them installed!

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras import layers

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

### Let's get started!

1\. **.**

2\. **.**

3\. **.**